In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from cycler import cycler
import requests
from pywaffle import Waffle
import numpy as np
import squarify
from textblob import TextBlob
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import math
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from matplotlib.pyplot import figure
from PIL import Image
from sklearn.cluster import KMeans
import matplotlib.colors as colors

%matplotlib inline



In [2]:
plt.rcParams.update({
    'pdf.fonttype': 42,
    'axes.prop_cycle': cycler('color', ['#52bbb7', '#8fdeb4', '#f16764', '#ca4862', '#461d2d', '#9C2964', '#1B7EBE', '#E4C34A']),
    'ps.fonttype': 42,
    'grid.linestyle': '--',
    'axes.facecolor': '#8fdeb4',
    'figure.facecolor': '#8fdeb4',
    'axes.spines.left' : True,
    'axes.spines.right' : False,
    'axes.spines.top' : False,
    'axes.spines.bottom' : True,
    'font.family': 'sans',
    'font.sans-serif': 'RobotoCondensed-Bold',
    'font.serif': 'Lora-Regular',
    'axes.titlesize': 'x-large',
    'axes.titleweight': 'bold',
    'axes.grid': True,
    'grid.color': 'silver',
    'axes.axisbelow': True,
    'xtick.bottom' : False,
    'ytick.left' : False,
    'axes.titlepad' : 15.0,
    'legend.frameon' : False,
    'ytick.labelsize': 15,
    'xtick.labelsize': 15,
    'axes.titlesize': 30,
    'text.color': '#1B7EBE',
    'axes.labelcolor': '#1B7EBE',
    'xtick.color': '#1B7EBE',
    'ytick.color': '#1B7EBE'
    
})

In [18]:
df = pd.read_csv('all-ufo-sightings-thru-08-06-2018.csv')

In [19]:
df.seen_at = pd.to_datetime(df.seen_at)

# WORD CLOUDS

In [ ]:
def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map"""

    # Note that if base_cmap is a string or None, you can simply do
    #    return plt.cm.get_cmap(base_cmap, N)
    # The following works for string, None, or a colormap instance:

    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)

In [ ]:
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

In [ ]:
default_stopwords = stopwords.words('english')

In [ ]:
arr = np.linspace(0, 50, 100).reshape((10, 10))
fig, ax = plt.subplots(ncols=2)

cmap = plt.get_cmap('bone')
new_cmap = truncate_colormap(cmap, 0.0, 0.4)
ax[0].imshow(arr, interpolation='nearest', cmap=cmap)
ax[1].imshow(arr, interpolation='nearest', cmap=new_cmap)
plt.show()

In [ ]:
'the' not in set(nltk.corpus.stopwords.words('english'))

In [ ]:
def make_decade_wordcloud(df):
    beam_mask = np.array(Image.open("beam-02.png"))

    description_list = df.text.values.tolist()

    word_list = []
    
    for description in description_list:
        tokenized_description = nltk.word_tokenize(description)
        for token in tokenized_description:
            word_list.append(token)

    default_stopwords = set(nltk.corpus.stopwords.words('english'))
    word_list = [word.lower() for word in word_list if (len(word) > 2) and (word.isalpha() == True) and (word not in default_stopwords) and (word != 'the') and (word != 'like')]
   # word_list = [word for word in word_list if word.isalpha() == True]
   # word_list = [word for word in word_list if word not in default_stopwords]
   # word_list = [word.lower() for word in word_list]
   # word_list = [word for word in word_list if word != '...']

    wordnet_lemmatizer = WordNetLemmatizer()

    lemma_list = [wordnet_lemmatizer.lemmatize(word) for word in word_list]

    fdist = nltk.FreqDist(lemma_list)

    unigram_wordcloud = WordCloud(colormap=new_cmap, height=300, relative_scaling=.32, width=400, max_words = 80, min_font_size=13, contour_width=2, contour_color='gray', background_color='#8fdeb4', font_path='/Users/kevinlitnav/Library/Fonts/RobotoCondensed-Regular.ttf', mask=beam_mask).generate_from_frequencies(fdist)
    
    fig, ax = plt.subplots(facecolor='#8fdeb4')

    figure(num=None, figsize=(12,8), edgecolor='#8fdeb4')

    plt.imshow(unigram_wordcloud, interpolation='bilinear')
    plt.grid(False)
    plt.xticks(np.arange(0))
    plt.yticks(np.arange(0))
    plt.tight_layout(pad=0)

    plt.savefig('word-cloud-decade.svg', facecolor=fig.get_facecolor(), transparent=True, bbox_inches='tight')
    
    return 

In [ ]:
df_1970s = df[df.seen_at.dt.year < 1980]
df_1980s = df[(df.seen_at.dt.year > 1979) & (df.seen_at.dt.year < 1990)]
df_1990s = df[(df.seen_at.dt.year > 1989) & (df.seen_at.dt.year < 2000)]
df_2000s = df[(df.seen_at.dt.year > 1999) & (df.seen_at.dt.year < 2010)]
df_2010s = df[df.seen_at.dt.year > 2009]

In [ ]:
description_list = df.text.values.tolist()

word_list = []
    
for description in description_list:
    tokenized_description = nltk.word_tokenize(description)
    for token in tokenized_description:
        word_list.append(token)

In [ ]:
default_stopwords = set(nltk.corpus.stopwords.words('english'))
#word_list = [word for word in word_list if len(word) > 2]
#word_list = [word for word in word_list if word.isalpha() == True]
word_list = [word for word in word_list if word not in default_stopwords]
#word_list = [word.lower() for word in word_list]
##word_list = [word for word in word_list if word != '...']
#word_list = [word for word in word_list if word != 'the']
#word_list = [word for word in word_list if word != 'like']

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
lemma_list = [wordnet_lemmatizer.lemmatize(word) for word in word_list]

In [ ]:
fdist = nltk.FreqDist(lemma_list)

In [ ]:
beam_mask = np.array(Image.open("beam-02.png"))

In [ ]:
unigram_wordcloud = WordCloud(colormap=new_cmap, height=300, relative_scaling=.32, width=400, max_words = 80, min_font_size=13, contour_width=2, contour_color='gray', background_color='#8fdeb4', font_path='/Users/kevinlitnav/Library/Fonts/RobotoCondensed-Regular.ttf', mask=beam_mask).generate_from_frequencies(fdist)

In [ ]:
fig, ax = plt.subplots(facecolor='#8fdeb4')

figure(num=None, figsize=(12,8), edgecolor='#8fdeb4')

plt.imshow(unigram_wordcloud, interpolation='bilinear')
plt.grid(False)
plt.xticks(np.arange(0))
plt.yticks(np.arange(0))
plt.tight_layout(pad=0)

plt.savefig('word-cloud.svg', facecolor=fig.get_facecolor(), transparent=True, bbox_inches='tight')


# UFO SHAPES

In [20]:
df['shape'].fillna('Unknown', inplace=True)

In [21]:
df.rename(columns={'shape':'craft_shape'},inplace=True)

In [22]:
df.craft_shape = df.craft_shape.str.replace('Unknown', 'unknown')

In [23]:
df.craft_shape = df.craft_shape.str.lower()

In [24]:
df_no_missing_times = df 

In [25]:
df_no_missing_times.dropna(subset=['seen_at'], inplace=True)

In [26]:
df_no_missing_times.shape

(114022, 13)

In [29]:
df_no_missing_times.year.value_counts()

2014    8671
2012    8092
2013    7799
2015    6883
2016    5624
2011    5608
2008    5292
2017    5255
2009    4982
2010    4768
2004    4737
2007    4730
2005    4519
2003    4413
2006    4160
2002    3685
2001    3530
1999    3153
2000    3092
1998    2020
2018    1628
1995    1472
1997    1438
1996    1016
1994     505
1978     405
1993     384
1975     383
1976     343
1977     339
1974     326
1990     316
1992     314
1979     306
1980     302
1989     302
1988     288
1973     284
1991     284
1987     278
1985     258
1986     233
1982     224
1984     216
1972     212
1981     210
1983     207
1970     196
1969     190
1971     146
1968       4
Name: year, dtype: int64

In [28]:
df_no_missing_times['year'] = df_no_missing_times.seen_at.dt.year

In [42]:
crafts_over_years = df_no_missing_times[(df_no_missing_times.craft_shape != 'unknown') & (df_no_missing_times.craft_shape != 'other')].groupby('year').craft_shape.value_counts(normalize=True).unstack()



In [32]:
crafts_over_years.shape

(51, 27)

In [ ]:
crafts_over_years.fillna(0).head()

In [ ]:
x,y = np.meshgrid(crafts_over_years.columns, crafts_over_years.index)

crafts_over_years *= 5000
plt.scatter(x=x.flatten(), y=y.flatten(), s=crafts_over_years.values.flatten())
plt.margins(.4)
plt.show()

In [43]:
crafts_over_years = crafts_over_years[['light', 'circle', 'oval', 'sphere', 'triangle', 'fireball', 'disk', 'formation', 'changing', 'cigar']].fillna(0)

In [45]:
crafts_over_years.to_csv('crafts_over_years.csv')

In [ ]:
df.rename(columns={'shape':'craft_shape'}, inplace=True)

In [ ]:
squarecolors = ['#52bbb7', '#8fdeb4', '#f16764', '#ca4862', '#461d2d', '#1B7EBE', '#E4C34A']

In [ ]:
#df_top.tonality_name.value_counts().plot(kind='barh')

plt.rcParams.update({'text.color': '#333333',
                    'axes.spines.left' : False,
                    'axes.spines.bottom' : False})

plt.rc('font', size=25)

fig, ax = plt.subplots(facecolor='#8fdeb4')

shape_labels = [shape.title() for shape in df[(df.craft_shape != 'unknown') & (df.craft_shape != 'other')].craft_shape.value_counts().head(10).index]

fig.set_size_inches(14,8)

squarecolors = ['#52bbb7', '#f16764', '#ca4862', '#9C2964', '#1B7EBE', '#E4C34A']

ax = squarify.plot(sizes=df[(df.craft_shape != 'unknown') & (df.craft_shape != 'other')].craft_shape.value_counts().head(10), label=shape_labels, color=squarecolors, alpha=.8)
#plt.axis('off')
#plt.show()

ax.grid(False)
ax.set_xticklabels(labels='')
ax.set_yticklabels(labels='')


#ax.set_title('UFO shapes', loc='left', pad=10, fontdict={'color': 'dimgray'}, size=30)
#ax.text(0,104, 'Of the 100 top rated tabs on Ultimate Guitar, the keys of C and G reign supreme.', fontdict={'color': '#DDDDC9', 'family': 'serif'}, size=20)
plt.savefig("ufo-shapes.svg", facecolor=fig.get_facecolor(), transparent=True, bbox_inches='tight')



# REGEXING

In [ ]:
df[df.text.str.contains('government')].iloc[1].text

In [ ]:
df[df.text.str.contains('Roswell|Area 51|chemtrails|new world order|')]

In [ ]:
df_no_missing_times.columns

In [ ]:
df_no_missing_times[df_no_missing_times.text.str.contains('Delonge')].groupby('year').text.count()

In [ ]:
df[df.text.str.contains('abduct')].text[111875]

In [ ]:
df[df.text.str.contains('NUFORC Note')]

In [ ]:
df[df.text.str.contains('skeptic')].iloc[120].text

In [ ]:
df[df.text.str.contains('Close Encounters')].text.iloc[4]

In [ ]:
df[df.text.str.contains(r'Independence Day')]

In [ ]:
df[df.text.str.contains(r'Art Bell')].iloc[2].text

In [ ]:
df[df.text.str.contains(r'abduct')].iloc[5].text

In [ ]:
df[df.text.str.contains('orbs')]

In [ ]:
df_no_missing_times[(df_no_missing_times.year != 2018) & (df_no_missing_times.text.str.contains(r'abduct'))].groupby('year').text.count().plot()

# Clustering

In [ ]:
def tokenize(s):
    blob = TextBlob(s.lower())
    words = [token for token in blob.words if token.isalpha() == True and len(token)>3]
    return words

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english', tokenizer=tokenize)

matrix = vectorizer.fit_transform(df.text)

In [ ]:
number_of_clusters = 5
km=KMeans(n_clusters=number_of_clusters)
km.fit(matrix)

In [ ]:
def print_sorted_vector(v):
    # this "lambda" thing is an anonymous function, google me to unluck bonus coding knowledge
    sorted_list = sorted(v.items(), key=lambda x: (x[1],x[0]), reverse=True) 
    sorted_list = sorted_list[:10]
    print('\n'.join([str(x) for x in sorted_list]))

In [ ]:
centroids = pd.DataFrame(km.cluster_centers_, columns=vectorizer.get_feature_names())
for i in range(number_of_clusters):
    print(f'\nCluster {i}')
    print_sorted_vector(centroids.iloc[i])

# Mapping

In [ ]:
df_geotagged = df.dropna(subset=['city_latitude', 'city_longitude'])

In [ ]:
df.state.value_counts()